# Polymarket Trading Activity Dump

This notebook fetches all trading activity from Polymarket and dumps it to a CSV file.

## Setup and Configuration

In [2]:
import os
import sys
from datetime import datetime, timedelta
import pandas as pd
import requests
from typing import List, Dict
from dotenv import load_dotenv

# Load environment variables
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
load_dotenv()

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## Configuration Parameters

Modify these parameters to control the time period and output location.

In [3]:
# ===== CONFIGURATION PARAMETERS =====

# Wallet address to analyze
WALLET_ADDRESS = os.getenv('BROWSER_ADDRESS')

# Time period configuration
# Option 1: Use relative time period (days ago)
DAYS_BACK = 2  # Fetch last N days of trading activity
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=DAYS_BACK)

# Option 2: Use specific dates (uncomment and modify if needed)
# START_DATE = datetime(2025, 10, 1)  # Year, Month, Day
# END_DATE = datetime(2025, 10, 27)

# Convert to Unix timestamps (seconds)
START_TIMESTAMP = int(START_DATE.timestamp())
END_TIMESTAMP = int(END_DATE.timestamp())

# API Configuration
API_BASE_URL = 'https://data-api.polymarket.com'
REQUEST_LIMIT = 500  # Max results per request

# Output file configuration
OUTPUT_CSV = f'trading_dump_{START_DATE.strftime("%Y%m%d")}_{END_DATE.strftime("%Y%m%d")}.csv'

print(f"Wallet Address: {WALLET_ADDRESS}")
print(f"Date Range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"Timestamp Range: {START_TIMESTAMP} to {END_TIMESTAMP}")
print(f"Output CSV: {OUTPUT_CSV}")

Wallet Address: 0xC3dA2b26f845d799B9d456828753f5a3af57d974
Date Range: 2025-10-27 to 2025-10-29
Timestamp Range: 1761525353 to 1761698153
Output CSV: trading_dump_20251027_20251029.csv


## Fetch Trading Activity from API

In [4]:
def fetch_trading_activity(
    wallet_address: str,
    start: int,
    end: int,
    activity_type: str = 'TRADE'
) -> List[Dict]:
    """
    Fetch all trading activity for a wallet within the specified timeframe.
    
    API Documentation: https://docs.polymarket.com/api-reference/core/get-user-activity
    
    Args:
        wallet_address: Ethereum wallet address
        start: Start timestamp (Unix seconds)
        end: End timestamp (Unix seconds)
        activity_type: Type of activity ('TRADE', 'DEPOSIT', 'WITHDRAWAL', etc.)
    
    Returns:
        List of trading activity records with all available fields
    """
    all_activities = []
    offset = 0
    
    print(f"Fetching {activity_type} activity from API...\n")
    
    while True:
        params = {
            'user': wallet_address,
            'type': activity_type,
            'limit': REQUEST_LIMIT,
            'offset': offset,
            'start': start,
            'end': end,
            'sortBy': 'TIMESTAMP',
            'sortDirection': 'DESC'
        }
        
        try:
            response = requests.get(f'{API_BASE_URL}/activity', params=params)
            response.raise_for_status()
            
            activities = response.json()
            
            if not activities or len(activities) == 0:
                break
            
            all_activities.extend(activities)
            
            print(f"  Fetched {len(activities)} activities (offset: {offset}, total: {len(all_activities)})")
            
            # Check if we've reached the end
            if len(activities) < REQUEST_LIMIT:
                break
            
            offset += REQUEST_LIMIT
            
        except requests.exceptions.RequestException as e:
            print(f"\n❌ Error fetching data: {e}")
            break
    
    print(f"\n✅ Total activities fetched: {len(all_activities)}\n")
    return all_activities

In [5]:
# Fetch all trading activity
activities = fetch_trading_activity(WALLET_ADDRESS, START_TIMESTAMP, END_TIMESTAMP)

if not activities:
    print("⚠️ No trading activity found in the specified timeframe")
else:
    print(f"✅ Successfully fetched {len(activities)} trades")
    print(f"\nAvailable fields in API response: {list(activities[0].keys())}")

Fetching TRADE activity from API...

  Fetched 103 activities (offset: 0, total: 103)

✅ Total activities fetched: 103

✅ Successfully fetched 103 trades

Available fields in API response: ['proxyWallet', 'timestamp', 'conditionId', 'type', 'size', 'usdcSize', 'transactionHash', 'price', 'asset', 'side', 'outcomeIndex', 'title', 'slug', 'icon', 'eventSlug', 'outcome', 'name', 'pseudonym', 'bio', 'profileImage', 'profileImageOptimized']


## Process and Export to CSV

In [6]:
# Convert to DataFrame
df = pd.DataFrame(activities)

# Add human-readable datetime
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

# Convert numeric fields
numeric_fields = ['usdcSize', 'size', 'price']
for field in numeric_fields:
    if field in df.columns:
        df[field] = pd.to_numeric(df[field], errors='coerce')

# Sort by timestamp (newest first)
df = df.sort_values('timestamp', ascending=False)

# Export to CSV with all available fields
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Exported {len(df)} trades to {OUTPUT_CSV}")
print(f"\nCSV contains {len(df.columns)} columns: {list(df.columns)}")
print("\nFirst few rows of the data:")
df.head(3)

✅ Exported 103 trades to trading_dump_20251027_20251029.csv

CSV contains 22 columns: ['proxyWallet', 'timestamp', 'conditionId', 'type', 'size', 'usdcSize', 'transactionHash', 'price', 'asset', 'side', 'outcomeIndex', 'title', 'slug', 'icon', 'eventSlug', 'outcome', 'name', 'pseudonym', 'bio', 'profileImage', 'profileImageOptimized', 'datetime']

First few rows of the data:


,proxyWallet,timestamp,conditionId,type,size,usdcSize,transactionHash,price,asset,side,outcomeIndex,title,slug,icon,eventSlug,outcome,name,pseudonym,bio,profileImage,profileImageOptimized,datetime
0,0xc3da2b26f845d799b9d456828753f5a3af57d974,1761696625,0xdb44407c2aa349c6b64662b6ba0d909e83d9b2371597...,TRADE,20.000000,5.800000,0x886ffe5cd357bc35db5a09cbae8851fd87caf8b05b59...,0.29,4543135460239246298424293339337949563883793477...,BUY,1,"Will Powell say ""Pandemic"" during October pres...",will-powell-say-pandemic-during-october-press-...,https://polymarket-upload.s3.us-east-2.amazona...,what-will-powell-say-during-october-press-conf...,No,alcaudi,Meaty-Prevalence,,,,2025-10-29 00:10:25
1,0xc3da2b26f845d799b9d456828753f5a3af57d974,1761689807,0xa840b73a86f96b0052e7420c5e7fdbfc6ddba53c0a86...,TRADE,20.000000,2.800000,0x90f3ef2ea2cff4559ac3d65dd84588b74fa6ef13d735...,0.14,8775860415663651356912192763756520376516333548...,SELL,1,"Will Powell say ""Inflation"" 40+ times during O...",will-powell-say-inflation-40-times-during-octo...,https://polymarket-upload.s3.us-east-2.amazona...,what-will-powell-say-during-october-press-conf...,No,alcaudi,Meaty-Prevalence,,,,2025-10-28 22:16:47
2,0xc3da2b26f845d799b9d456828753f5a3af57d974,1761689775,0xd1796c09d0d6f876f8580086ae9808ec991784e3a74b...,TRADE,26.315788,16.315789,0xbbf78feeb1ebf0b43daae42bd2aab64228f7db2394d6...,0.62,1146947264513076545289485589678984936629170706...,BUY,0,Netanyahu out by end of 2026?,netanyahu-out-before-2027,https://polymarket-upload.s3.us-east-2.amazona...,netanyahu-out-before-2027,Yes,alcaudi,Meaty-Prevalence,,,,2025-10-28 22:16:15


## Display Trades by Market

Group trades by market (conditionId) and display them sorted by total volume.

In [7]:
# Group by market (conditionId) and calculate total volume
market_summary = df.groupby(['conditionId', 'title']).agg({
    'usdcSize': 'sum',
    'timestamp': 'count'
}).reset_index()

market_summary.columns = ['conditionId', 'title', 'total_volume', 'trade_count']

# Sort by total volume (descending)
market_summary = market_summary.sort_values('total_volume', ascending=False)

print("📊 MARKET SUMMARY")
print("=" * 120)
print(f"\nTotal Markets Traded: {len(market_summary)}")
print(f"Total Volume: ${market_summary['total_volume'].sum():,.2f}")
print(f"Total Trades: {market_summary['trade_count'].sum():,.0f}")
print("\n" + "=" * 120)

market_summary

📊 MARKET SUMMARY

Total Markets Traded: 33
Total Volume: $4,243.12
Total Trades: 103



,conditionId,title,total_volume,trade_count
21,0xa4bdb50db073da1c1ea7bbebe1a1fda29ada113f9e86...,Will the U.S. tariff rate on China be between ...,943.341200,12
17,0x8195ceb3ab6fd94b2adf05993501a4ec9a95dd5eca25...,Will the Government shutdown end by November 30?,754.999200,12
2,0x1d9d1019dc213e5e76eaa87743b7ac2449a2121aabf4...,US x China tariff agreement by November 10?,362.000000,2
9,0x3f8f28bade4519d43d4ccd7e11311e71998ec535887c...,Will House and Senate pass funding bill by Nov...,295.715000,6
3,0x21455b5d18026dbb4564366ecb7390b8d88099285713...,Will Democrats win five elections in Nov?,253.340800,12
14,0x77eaf36a80197a13ff32e31f05b54e824a6fd21d93f1...,Will 'A Minecraft Movie' have the best domesti...,180.069500,4
24,0xb85b0091c8839bc4d4fbd6401f044474d73235ac81d8...,Will Savannah Louie win Survivor season 49?,171.560000,3
20,0xa34edb6c898232378a9cc08744842df2cd89d278ba45...,Will Lamine Yamal win the 2026 Ballon d'Or?,139.062000,2
4,0x21d5b50fd0bc5013be79a7cdb4b3f0917127071cf27a...,Will Trump refrain from announcing a next Fed ...,120.940000,3
1,0x1d039941b393f61a08567d37ef2e45bc3e7b19984d18...,Will Trump establish a Gaza “Board of Peace” i...,113.526400,2


## Detailed Trades by Market (Sorted by Volume)

In [10]:
def display_market_trades(df: pd.DataFrame, market_summary: pd.DataFrame):
    """
    Display all trades for each market, sorted by total market volume.
    
    Args:
        df: DataFrame with all trades
        market_summary: DataFrame with market summaries sorted by volume
    """
    for idx, market in market_summary.iterrows():
        condition_id = market['conditionId']
        
        # Get all trades for this market
        market_trades = df[df['conditionId'] == condition_id].copy()
        market_trades = market_trades.sort_values('timestamp', ascending=True)
        
        # Display market header
        print("\n" + "=" * 120)
        print(f"MARKET #{idx + 1}: {market['title']}")
        print("=" * 120)
        print(f"Condition ID: {condition_id}")
        print(f"Total Volume: ${market['total_volume']:,.2f}")
        print(f"Total Trades: {market['trade_count']:.0f}")
        print("\n" + "-" * 120)
        
        # Display each trade
        for trade_idx, trade in market_trades.iterrows():
            # Get outcome/side information
            outcome = trade.get('outcome', 'N/A')
            side = trade.get('side', 'N/A')
            
            print(f"\n[{trade['datetime'].strftime('%Y-%m-%d %H:%M:%S')}] {side:4s} | Outcome: {outcome}")
            print(f"  Trade: {trade['size']:,.2f} tokens @ ${trade['price']:.4f} = ${trade['usdcSize']:,.2f} USDC")
        
        print("\n" + "-" * 120)

# Display all trades grouped by market
display_market_trades(df, market_summary)


MARKET #22: Will the U.S. tariff rate on China be between 25% and 40% on November 12?
Condition ID: 0xa4bdb50db073da1c1ea7bbebe1a1fda29ada113f9e863dc2cd3f37b2295ce4b7
Total Volume: $943.34
Total Trades: 12

------------------------------------------------------------------------------------------------------------------------

[2025-10-27 12:18:55] BUY  | Outcome: Yes
  Trade: 158.94 tokens @ $0.7500 = $119.20 USDC

[2025-10-27 12:24:39] BUY  | Outcome: Yes
  Trade: 20.00 tokens @ $0.7500 = $15.00 USDC

[2025-10-27 12:25:35] BUY  | Outcome: Yes
  Trade: 40.00 tokens @ $0.7500 = $30.00 USDC

[2025-10-27 12:25:35] BUY  | Outcome: Yes
  Trade: 10.00 tokens @ $0.7500 = $7.50 USDC

[2025-10-27 12:25:35] BUY  | Outcome: Yes
  Trade: 48.65 tokens @ $0.7500 = $36.49 USDC

[2025-10-27 14:30:47] SELL | Outcome: Yes
  Trade: 93.51 tokens @ $0.7500 = $70.13 USDC

[2025-10-27 19:01:15] BUY  | Outcome: Yes
  Trade: 4.54 tokens @ $0.7300 = $3.31 USDC

[2025-10-27 19:04:35] BUY  | Outcome: Yes
  Trad

## Summary Statistics

In [ ]:
# Trading statistics
buy_trades = df[df['side'] == 'BUY']
sell_trades = df[df['side'] == 'SELL']

print("📈 TRADING STATISTICS")
print("=" * 80)
print(f"\nTime Period: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"Wallet: {WALLET_ADDRESS}")
print(f"\nTotal Trades: {len(df)}")
print(f"  - BUY:  {len(buy_trades)} trades (${buy_trades['usdcSize'].sum():,.2f} USDC)")
print(f"  - SELL: {len(sell_trades)} trades (${sell_trades['usdcSize'].sum():,.2f} USDC)")
print(f"\nTotal Volume: ${df['usdcSize'].sum():,.2f} USDC")
print(f"Average Trade Size: ${df['usdcSize'].mean():,.2f} USDC")
print(f"Median Trade Size: ${df['usdcSize'].median():,.2f} USDC")
print(f"\nMarkets Traded: {df['conditionId'].nunique()}")
print(f"Unique Outcomes Traded: {df['asset'].nunique()}")
print(f"\nData exported to: {OUTPUT_CSV}")
print("\n" + "=" * 80)